In [1]:
import numpy as np
from keras.layers import Dense, Flatten, Dropout, Input, BatchNormalization
from keras.utils import np_utils
from keras.models import Sequential, Model
from keras import regularizers
import torch
from keras import optimizers
from keras.callbacks import ModelCheckpoint
from keras.layers.merge import concatenate
from keras import initializers
from keras.utils import to_categorical
from keras.layers.advanced_activations import LeakyReLU, PReLU

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
import threading
os.environ["CUDA_VISIBLE_DEVICES"]="2"
import pickle
import gc

import keras
import tensorflow as tf

Using TensorFlow backend.


In [2]:
import pickle
with open('gossip/finalTrainEmbeddings.pkl', 'rb') as f:
    trainEmbeddings = pickle.load(f)
with open('gossip/finalTestEmbeddings.pkl', 'rb') as f:
    testEmbeddings = pickle.load(f)

In [3]:
import json
with open('imageDataset/gossip/trainJson.json', 'r') as f:
    trainData = json.load(f)
with open('imageDataset/gossip/testJson.json', 'r') as f:
    testData = json.load(f)

In [4]:
for i in trainEmbeddings:
    if len(trainEmbeddings[i]) >=50:
        trainEmbeddings[i] = trainEmbeddings[i][0:50]
    else:
        deficit = 50 - len(trainEmbeddings[i])
        for j in range(deficit):
            trainEmbeddings[i].append(np.zeros((1,768), dtype=np.float32))
    temp = np.empty((50,768), dtype=np.float32)
    for j in range(len(trainEmbeddings[i])):
        temp[j][:] = trainEmbeddings[i][j]
    trainEmbeddings[i] = temp

In [5]:
with open('gossip/train_gossip.pickle', 'rb') as f:
    train_vgg_poli = pickle.load(f)
with open('gossip/test_gossip.pickle', 'rb') as f:
    test_vgg_poli = pickle.load(f)

In [6]:
for i in testEmbeddings:
    if len(testEmbeddings[i]) >=50:
        testEmbeddings[i] = testEmbeddings[i][0:50]
    else:
        deficit = 50 - len(testEmbeddings[i])
        for j in range(deficit):
            testEmbeddings[i].append(np.zeros((1,768), dtype=np.float32))
    temp = np.empty((50,768), dtype=np.float32)
    for j in range(len(testEmbeddings[i])):
        temp[j][:] = testEmbeddings[i][j]
    testEmbeddings[i] = temp

In [7]:
train_text = [] # text embeddings
train_label = [] # labels
test_text = [] # text embeddings
test_label = []
train_image = [] # image embeddings
test_image = []

# Train Image IDs
# Test Image IDs
trainImageNames = [] # names of the images i.e name.jpg
trainTextNames = []  # train articles
testTextNames = []   # test articles
testImageNames = []  # names of the images in the test folder

In [8]:
# note: finalTrainEmbeddings.pkl---> trainDataJson.json----> trainTextNames
# trainDataJson.json is a dictionary: format---> key (article i.e title+text): [label, article_url, imageId]
# In traindataJson.json , we have complete list of data with all the attributes and trainTextnames contains all the samples that have associated images with it. this is obtained by running the "Match code" given below.

In [9]:
# match code
# Check whether Image exists, then get corresponding Text Embeddings, and finally append to respective lists
for i in train_vgg_poli:
    for j in trainData:
        if i.split('.jpg')[0] == trainData[j][-1]['Top_img']:
            if j in trainEmbeddings:
                trainImageNames.append(i)
                trainTextNames.append(j)
                train_text.append(trainEmbeddings[j])
                train_image.append(train_vgg_poli[trainData[j][-1]['Top_img'] + '.jpg'])
                train_label.append(trainData[j][0])

In [10]:
for i in test_vgg_poli:
    for j in testData:
        if i.split('.jpg')[0] == testData[j][-1]['Top_img']:
            if j in testEmbeddings:
                testImageNames.append(i)
                testTextNames.append(j)
                test_text.append(testEmbeddings[j])
                test_image.append(test_vgg_poli[testData[j][-1]['Top_img'] + '.jpg'])
                test_label.append(testData[j][0])

In [15]:
len(trainTextNames), len(testTextNames)

(10010, 2830)

In [16]:
len(trainImageNames), len(testImageNames)

(10010, 2830)

In [17]:
len(train_label), len(test_label)

(10010, 2830)

In [18]:
# to count total number of real and fake articles in train and test split
realCount = 0
fakeCount = 0
for i in train_label:
    if i == 1:
        realCount += 1
    elif i == 0:
        fakeCount += 1

for i in test_label:
    if i == 1:
        realCount += 1
    elif i == 0:
        fakeCount += 1
print(realCount, fakeCount)

10259 2581


In [16]:
# # saving updated final dataset in the csv files for future reference
# import pandas as pd
# df=pd.DataFrame()
# # df['train_articles']=trainTextNames
# # df.to_csv('gossip_train_articles.csv', index=False)

# # df['test_articles']=testTextNames
# # df.to_csv('gossip_test_articles.csv', index=False)

# df['train_label']=train_label
# df.to_csv('gossip_train_label.csv', index=False)

# # df['test_label']=test_label
# # df.to_csv('gossip_test_label.csv', index=False)

# # df['train_image']=trainImageNames
# # df.to_csv('gossip_train_image.csv', index=False)

# # df['test_image']=testImageNames
# # df.to_csv('gossip_test_image.csv', index=False)


In [19]:
train_label = to_categorical(train_label)
test_label = to_categorical(test_label)

In [20]:
"""train_text=[torch.Tensor.numpy(i.cpu()) for i in train_text]
test_text=[torch.Tensor.numpy(i.cpu()) for i in test_text]"""
train_image[0].shape

(1, 4096)

In [21]:
train_text_matrix = np.ndarray(shape=(len(train_text), 50,768))

In [22]:
counter = 0
for i in train_text:
    train_text_matrix[counter][:][:] = i
    counter += 1

In [23]:
test_text_matrix = np.ndarray(shape=(len(test_text), 50,768))

In [24]:
counter = 0
for i in test_text:
    test_text_matrix[counter][:][:] = i
    counter += 1

In [25]:
test_text_matrix.shape

(2830, 50, 768)

In [26]:
train_image[0].shape

(1, 4096)

In [27]:
train_image_matrix = np.ndarray(shape=(len(train_image), 4096,1))

In [28]:
counter = 0
for i in train_image:
    train_image_matrix[counter][:][:] = i.reshape(4096,1)
    counter += 1

In [29]:
test_image_matrix = np.ndarray(shape=(len(test_image), 4096,1))

In [30]:
counter = 0
for i in test_image:
    test_image_matrix[counter][:][:] = i.reshape(4096,1)
    counter += 1

In [31]:
len(train_image_matrix)

10010

In [32]:
train_image_matrix = train_image_matrix.reshape(len(train_image_matrix),4096)
test_image_matrix = test_image_matrix.reshape(len(test_image_matrix),4096)

### XLNET+dense+VGG

In [33]:
input_text = Input(shape=(50,768))
text_flat = Flatten()(input_text)
dense_text = Dense(1000,activation='relu',kernel_regularizer=regularizers.l2(0.01), kernel_initializer=initializers.he_normal(seed=0))(text_flat)
#dense_text = Dropout(0.4)(dense_text)
dense_text = Dense(500,activation='relu',kernel_regularizer=regularizers.l2(0.01), kernel_initializer=initializers.he_normal(seed=0))(dense_text)
#dense_text = Dropout(0.4)(dense_text)
dense_text = Dense(100,activation='relu',kernel_regularizer=regularizers.l2(0.01), kernel_initializer=initializers.he_normal(seed=0))(dense_text)
dense_text = BatchNormalization()(dense_text)
dense_text_drop = Dropout(0.4)(dense_text)

input_image = Input(shape=(4096,))
dense_image = Dense(2000,activation='relu',kernel_regularizer=regularizers.l2(0.01), kernel_initializer=initializers.he_normal(seed=0))(input_image)
#dense_image = Dropout(0.4)(dense_image)
dense_image = Dense(1000, activation='relu',kernel_regularizer=regularizers.l2(0.01), kernel_initializer=initializers.he_normal(seed=0))(dense_image)
#dense_image = Dropout(0.4)(dense_image)
dense_image = Dense(100,activation='relu',kernel_regularizer=regularizers.l2(0.01), kernel_initializer=initializers.he_normal(seed=0))(dense_image)
dense_image = BatchNormalization()(dense_image)
dense_image_drop = Dropout(0.4)(dense_image)

concat = concatenate([dense_text_drop,dense_image_drop])

inter1_dense = Dense(200,activation='relu',kernel_regularizer=regularizers.l2(0.01), kernel_initializer=initializers.he_normal(seed=0))(concat)
inter1_dense = Dense(100,activation='relu',kernel_regularizer=regularizers.l2(0.01), kernel_initializer=initializers.he_normal(seed=0))(inter1_dense)
final_dense = Dense(50,activation='relu',kernel_regularizer=regularizers.l2(0.01), kernel_initializer=initializers.he_normal(seed=0))(inter1_dense)
final_dropout = Dropout(0.4)(final_dense)
output = Dense(2, activation='softmax')(final_dropout)

model = Model(inputs=[input_text,input_image], outputs=output)
adam = optimizers.Adam(lr=1e-4)
#adagrad = optimizers.Adagrad(lr=1e-4)
#adamax = optimizers.Adamax(lr=0.002, beta_1=0.9, beta_2=0.999)
checkpoint = ModelCheckpoint(filepath='../checkpoints_gossip/dense_MM_model.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

#sgd = optimizers.SGD(lr=1e-4, clipnorm=1.)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 50, 768)      0                                            
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 38400)        0           input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 4096)         0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 1000)         38401000    flatten_1[0][0]                  
____________________________________________________________________________________________

In [34]:
history = model.fit([train_text_matrix, train_image_matrix],train_label,validation_data=([test_text_matrix,test_image_matrix],test_label),batch_size =32,epochs=100,callbacks=callbacks_list)

Train on 10010 samples, validate on 2830 samples
Epoch 1/100
10010/10010 [==============================] - 15s 1ms/step - loss: 73.6526 - accuracy: 0.7553 - val_loss: 54.0207 - val_accuracy: 0.8131
Epoch 2/100
  160/10010 [..............................] - ETA: 11s - loss: 53.8265 - accuracy: 0.7937

/home/shivangi/.conda/envs/BERT/lib/python3.7/site-packages/keras/callbacks/callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


10010/10010 [==============================] - 13s 1ms/step - loss: 42.0623 - accuracy: 0.7956 - val_loss: 32.1249 - val_accuracy: 0.8134
Epoch 3/100
10010/10010 [==============================] - 13s 1ms/step - loss: 25.4229 - accuracy: 0.8054 - val_loss: 19.8380 - val_accuracy: 0.8265
Epoch 4/100
10010/10010 [==============================] - 13s 1ms/step - loss: 15.9667 - accuracy: 0.8267 - val_loss: 12.7478 - val_accuracy: 0.8170
Epoch 5/100
10010/10010 [==============================] - 13s 1ms/step - loss: 10.4579 - accuracy: 0.8392 - val_loss: 8.5801 - val_accuracy: 0.8269
Epoch 6/100
10010/10010 [==============================] - 13s 1ms/step - loss: 7.1422 - accuracy: 0.8489 - val_loss: 5.9884 - val_accuracy: 0.8329
Epoch 7/100
10010/10010 [==============================] - 13s 1ms/step - loss: 5.0724 - accuracy: 0.8554 - val_loss: 4.3518 - val_accuracy: 0.8484
Epoch 8/100
10010/10010 [==============================] - 13s 1ms/step - loss: 3.7515 - accuracy: 0.8635 - val_loss:

10010/10010 [==============================] - 13s 1ms/step - loss: 0.0969 - accuracy: 0.9938 - val_loss: 0.8730 - val_accuracy: 0.8226
Epoch 58/100
10010/10010 [==============================] - 13s 1ms/step - loss: 0.1047 - accuracy: 0.9913 - val_loss: 0.8248 - val_accuracy: 0.8283
Epoch 59/100
10010/10010 [==============================] - 13s 1ms/step - loss: 0.0988 - accuracy: 0.9924 - val_loss: 0.8597 - val_accuracy: 0.8371
Epoch 60/100
10010/10010 [==============================] - 13s 1ms/step - loss: 0.0926 - accuracy: 0.9941 - val_loss: 0.8608 - val_accuracy: 0.8375
Epoch 61/100
10010/10010 [==============================] - 13s 1ms/step - loss: 0.0987 - accuracy: 0.9922 - val_loss: 0.8661 - val_accuracy: 0.8343
Epoch 62/100
10010/10010 [==============================] - 13s 1ms/step - loss: 0.0930 - accuracy: 0.9936 - val_loss: 0.8099 - val_accuracy: 0.8364
Epoch 63/100
10010/10010 [==============================] - 13s 1ms/step - loss: 0.1098 - accuracy: 0.9876 - val_loss: 

In [36]:
with open('XL_gossip_history.json', 'w') as f:
    json.dump(str(history.history), f)

In [34]:
import matplotlib.pyplot as plt
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

KeyError: 'acc'

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

### XLNET + dense

In [ ]:
def baseline_model():
    model = Sequential()
    model.add(Flatten(input_shape=(50,768)))
    model.add(Dense(1000, activation='relu', kernel_regularizer=regularizers.l2(0.01), kernel_initializer=initializers.he_normal(seed=0)))
    #model.add(Dropout(0.4))
    model.add(Dense(500, activation='relu', kernel_regularizer=regularizers.l2(0.01), kernel_initializer=initializers.he_normal(seed=0)))
    #model.add(Dropout(0.4))
    model.add(Dense(100, activation='relu', kernel_regularizer=regularizers.l2(0.01), kernel_initializer=initializers.he_normal(seed=0)))
    #model.add(BatchNormalization())
    model.add(Dropout(0.4))
    model.add(Dense(2, activation='softmax'))
    #adam = optimizers.Adam(lr=1e-4)
    #adamax = optimizers.Adamax(learning_rate=0.002, beta_1=0.9, beta_2=0.999)
    sgd = optimizers.SGD(lr=0.01, clipnorm=1.)
    #adadelta = optimizers.Adadelta(lr=1.0, rho=0.95)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

In [ ]:
checkpoint = ModelCheckpoint(filepath='/home/karmanya/finalExperiment/checkpoints_gossip/dense_Text_model.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
model = baseline_model()
model.summary()

In [ ]:
textHistory = model.fit(train_text_matrix,train_label,validation_data=(test_text_matrix,test_label),batch_size =32,epochs =100,callbacks=callbacks_list)

In [ ]:
"""import matplotlib.pyplot as plt
plt.plot(textHistory.history['acc'])
plt.plot(textHistory.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()"""

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D,MaxPooling1D,Flatten
from keras.layers import LSTM, Bidirectional
from keras.optimizers import RMSprop

### XLNET+ LSTM

In [ ]:
def lstm():
    model = Sequential()
    model.add(LSTM(128,input_shape=(50,768)))
    model.add(Dropout(0.5))
    model.add(Dense(595, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])
    return model

In [ ]:
checkpoint = ModelCheckpoint(filepath='/home/karmanya/finalExperiment/checkpoints_gossip/lstm_text_model.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
model = lstm()

In [ ]:
model.summary()

In [ ]:
model.fit(train_text_matrix,train_label,validation_data=(test_text_matrix,test_label),batch_size =32,epochs =100,callbacks=callbacks_list)

### XLNET + CNN

In [ ]:
def cnn_model():
    model = Sequential()
    model.add(Conv1D(filters=3, kernel_size=5, activation='relu',data_format='channels_first' , input_shape=(50,768)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=3, kernel_size=5, activation='relu',data_format='channels_first' ))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(500, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    return model

In [ ]:
model=cnn_model()
checkpoint = ModelCheckpoint(filepath='/home/karmanya/finalExperiment/checkpoints_gossip/cnn_text_model.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
model.summary()

In [ ]:
model.fit(train_text_matrix,train_label,validation_data=(test_text_matrix,test_label),batch_size =32,epochs =100, callbacks=callbacks_list)